In [1]:
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds


# Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name="mnist",with_info = True, as_supervised = True)

In [3]:
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

In [4]:
num_validation_samples = .1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [6]:
scaled_mnist_train_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [7]:
buffer_size = 10000

shuffled_train_validation_data = scaled_mnist_train_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data = shuffled_train_validation_data.skip(num_validation_samples)

#Hyperparamter that can be changed
batch_size = 100

In [8]:
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_input, validation_target = next(iter(validation_data))

# Model 


In [12]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                    tf.keras.layers.Flatten(input_shape=(28,28,1)),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(output_size,activation = 'softmax')   
        ])

# Optimizer algorithm and objective function 

In [13]:
#Optimizer and loss function
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

# Training


In [15]:
epochs = 10

model.fit(train_data,epochs=epochs,validation_data=(validation_input,validation_target),validation_steps=10,verbose=2)

Epoch 1/10
540/540 - 13s - loss: 0.0520 - accuracy: 0.9840 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
540/540 - 14s - loss: 0.0459 - accuracy: 0.9853 - val_loss: 0.0496 - val_accuracy: 0.9848
Epoch 3/10
540/540 - 15s - loss: 0.0380 - accuracy: 0.9879 - val_loss: 0.0475 - val_accuracy: 0.9860
Epoch 4/10
540/540 - 15s - loss: 0.0345 - accuracy: 0.9885 - val_loss: 0.0443 - val_accuracy: 0.9862
Epoch 5/10
540/540 - 15s - loss: 0.0307 - accuracy: 0.9901 - val_loss: 0.0539 - val_accuracy: 0.9848
Epoch 6/10
540/540 - 15s - loss: 0.0309 - accuracy: 0.9899 - val_loss: 0.0548 - val_accuracy: 0.9832
Epoch 7/10
540/540 - 15s - loss: 0.0261 - accuracy: 0.9921 - val_loss: 0.0385 - val_accuracy: 0.9865
Epoch 8/10
540/540 - 15s - loss: 0.0268 - accuracy: 0.9913 - val_loss: 0.0292 - val_accuracy: 0.9893
Epoch 9/10
540/540 - 15s - loss: 0.0231 - accuracy: 0.9927 - val_loss: 0.0370 - val_accuracy: 0.9887
Epoch 10/10
540/540 - 15s - loss: 0.0212 - accuracy: 0.9932 - val_loss: 0.0281 - va

# Testing

In [19]:
test_loss, test_accuracy = model.evaluate(test_data)

print(f'Test Loss: {test_loss}, Test Accuracy: {100*test_accuracy}%')

1/1 [==============================] - 1s 1s/step - loss: 0.1052 - accuracy: 0.9768
Test Loss: 0.10524673014879227, Test Accuracy: 97.680002450943%
